In [3]:
cd '/home/user/project/python/rmsKit/'

/home/user/project/python/rmsKit


In [4]:

from importlib import reload
import sys
sys.path.append("/home/user/project/python/rmsKit")
import rms

import jax 
import numpy as np
import jax.numpy as jnp
from jax.config import config
sys.path.append("/home/user/project/python/reduce_nsp")
sys.path.append("/home/user/project/python/exact")
import numpy as np

import os
groundstate = np.load("../exact/test/out/KH_2x2/Jx_1_Jy_1_Jz_1_h_0/groundstate.npy")
# os.environ["CUDA_VISIBLE_DEVICES"] = ""
# os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"


ImportError: /home/user/miniconda3/envs/py39/lib/python3.9/site-packages/torch/lib/libtorch_cuda.so: undefined symbol: cudaGraphDebugDotPrint, version libcudart.so.11.0

In [ ]:
import rms
reload(rms)

<module 'rms' from '/home/user/project/python/rmsKit/rms/__init__.py'>

In [ ]:
ur = rms.unitary.UnitaryRiemanGenerator(8, jax.random.PRNGKey(0), np.float64)
u = ur.reset_matrix()

UnitaryRiemanGenerator is initialized


In [ ]:
import sys
sys.path.append("/home/user/project/python/rmsKit")
from jax_lattice import KH

p = dict(
    Jx=1,
    Jy=1,
    Jz=1,
    hx=0,
    hz=0,
)
H = KH.system([2, 2], "3site", p)

In [ ]:
import numpy as np
u = np.load("array/KH/3site/sel/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_1/u/0.npy")
# U = np.kron(u,u)
# U = np.kron(U,U)

In [ ]:
state = rms.loss.init_loss(jnp.array(H), 8, np.float64, "sel", beta=1.0)
state_list = [state]
qesLoss = rms.loss.system_el_multi

In [ ]:
def loss(u):
    return qesLoss(state_list, jnp.array(u))

In [ ]:
%timeit loss(jnp.array(u)).block_until_ready()

In [ ]:
%%timeit 
v, g = jax.value_and_grad(loss)(jnp.array(u))
v.block_until_ready(), g.block_until_ready()

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("/home/user/project/python/rmsKit")
import os
import torch
from jax_lattice import KH
# # os.environ["CUDA_VISIBLE_DEVICES"] = ""
# # os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
import rms_torch
import numpy as np
u = np.load("/home/user/project/python/rmsKit/array/KH/3site/sel/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_1/u/0.npy")
device = torch.device("cuda")

/home/user/miniconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

p = dict(
    Jx=1,
    Jy=1,
    Jz=1,
    hx=0,
    hz=0,
)
H = KH.system([2, 2], "3site", p)

# Create an instance of the CustomModel class
H_gpu = torch.tensor(H, dtype=torch.float64, device=device)
model = rms_torch.CustomModel(H.shape[0], 8, device=device).to(device)
loss = rms_torch.CustomLoss(H, device=device).to(device)
compiled_model = torch.compile(model, dynamic = False, fullgraph=True)
compiled_loss = torch.compile(loss, dynamic = False, fullgraph=True)


In [6]:
optimizer = rms_torch.Adam(compiled_model.parameters(), lr=1e-2, amsgrad=True)
optimizer = rms_torch.LION(compiled_model.parameters(), lr=1e-2)
num_epochs = 1000
for epoch in range(num_epochs):
    # Zero out the gradients
    optimizer.zero_grad()

    # Forward pass
    output = compiled_model()
    loss = compiled_loss(output)
    loss.backward()
    for p in model.parameters():
        p.grad.data[:] = rms_torch.riemannian_grad_torch(p.data, p.grad.data)
    optimizer.step()

    print(f"Epoch: {epoch}, Loss: {loss.item()}")

Epoch: 0, Loss: 22.475547796416272
Epoch: 1, Loss: 22.21875121602225
Epoch: 2, Loss: 21.954006576007618
Epoch: 3, Loss: 21.68828910220547
Epoch: 4, Loss: 21.422626939625374
Epoch: 5, Loss: 21.155914233097086
Epoch: 6, Loss: 20.89628964831522
Epoch: 7, Loss: 20.637212004257908
Epoch: 8, Loss: 20.38549971116649
Epoch: 9, Loss: 20.127650674900377
Epoch: 10, Loss: 19.874725505362687
Epoch: 11, Loss: 19.619547875814405
Epoch: 12, Loss: 19.368795238926275
Epoch: 13, Loss: 19.11119331292672
Epoch: 14, Loss: 18.851705484450164
Epoch: 15, Loss: 18.59491850611301
Epoch: 16, Loss: 18.34478995877359
Epoch: 17, Loss: 18.093933759107863
Epoch: 18, Loss: 17.854392942134105
Epoch: 19, Loss: 17.647696591429263
Epoch: 20, Loss: 17.423194096339227
Epoch: 21, Loss: 17.21861332760075
Epoch: 22, Loss: 16.986561727970724
Epoch: 23, Loss: 16.752122278953664
Epoch: 24, Loss: 16.549372857474125
Epoch: 25, Loss: 16.323481466094517
Epoch: 26, Loss: 16.148867400897608
Epoch: 27, Loss: 15.913111157432441
Epoch: 28,

KeyboardInterrupt: 

In [ ]:

from importlib import reload
import sys
sys.path.append("/home/user/project/python/rmsKit")
import rms

import jax 
import numpy as np
import jax.numpy as jnp

sys.path.append("/home/user/project/python/reduce_nsp")
sys.path.append("/home/user/project/python/exact")
import numpy as np

import os
groundstate = np.load("/home/user/project/python/exact/test/out/KH_2x2/Jx_1_Jy_1_Jz_1_h_0/groundstate.npy")
u = np.load("/home/user/project/python/rmsKit/array/KH/3site/sel/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_1/u/0.npy")
H0 = -np.load("/home/user/project/python/rmsKit/array/KH/3site/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/0.npy")
H1 = -np.load("/home/user/project/python/rmsKit/array/KH/3site/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/1.npy")
H2 = -np.load("/home/user/project/python/rmsKit/array/KH/3site/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/2.npy")
x1 = groundstate.copy()
x0 = groundstate.reshape([8] * 4)
x0 = x0.transpose([0, 2, 1, 3]).reshape(-1)
x0 = jnp.array(x0)
# os.environ["CUDA_VISIBLE_DEVICES"] = ""
# os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
SH0 = rms.sum_ham(H0, [[0,2]], 4, 8)
SH1 = rms.sum_ham(H1, [[0,1]], 4, 8)
SH2 = rms.sum_ham(H2, [[0,3]], 4, 8)


In [ ]:
_bonds = [[0,0,2], [1,0,1], [2,0,3], [0,1,3], [1,1,0], [2,1,2], [0,2,0], [1,2,3], 
            [2,2,1], [0,3,1], [1,3,2], [2,3,0]]
bonds = [[], [], []]
for bond in _bonds:
    bonds[bond[0]].append(bond[1:])

_H = rms.sum_ham(rms.stoquastic(H0), bonds[0], 4, 8)
_H += rms.sum_ham(rms.stoquastic(H1), bonds[1], 4, 8)
_H += rms.sum_ham(rms.stoquastic(H2), bonds[2], 4, 8)

u2 = np.kron(u, u)
_HU = rms.sum_ham(rms.stoquastic(u2@H0@u2.T), bonds[0], 4, 8)
_HU += rms.sum_ham(rms.stoquastic(u2@H1@u2.T), bonds[1], 4, 8)
_HU += rms.sum_ham(rms.stoquastic(u2@H2@u2.T), bonds[2], 4, 8)

In [ ]:
U = np.kron(u,u)
U = np.kron(U,U)

SH = SH0 + SH1 + SH2

x = jnp.abs(U @ groundstate)
print(x @ _HU @ x)
x = jnp.abs(groundstate)
print(x @ rms.stoquastic(H) @ x)
# print(groundstate @ rms.stoquastic(H) @ groundstate)

# jnp.linalg.eigvalsh(rms.stoquastic(U @ H @ U.T))

-7.052132213843223
-6.22515992356388


In [ ]:
1.763032714130139 * 4

7.052130856520556

In [ ]:
np.linalg.norm(_H - rms.stoquastic(H))

1.579816545884742e-14

In [ ]:
jnp.linalg.eigvalsh(rms.stoquastic(H))

Array([-8.19503796, -7.8031308 , -7.8031308 , ...,  5.        ,
        6.        ,  6.        ], dtype=float64)

In [ ]:
jnp.linalg.eigvalsh(_H)

Array([-8.19503796, -7.8031308 , -7.8031308 , ...,  5.        ,
        6.        ,  6.        ], dtype=float64)

In [ ]:
jnp.linalg.eigvalsh(H)

Array([-5.44487522, -5.3283924 , -5.29823654, ...,  6.        ,
        6.        ,  6.        ], dtype=float64)

In [ ]:
state = rms.loss.init_loss(jnp.array(H1), 8, np.float64, "qes", X=jnp.array(x1))
qesLoss = rms.loss.qes_multi

qesLoss([state], jnp.array(u))

Array(0.52065724, dtype=float64)

## With Bug

In [ ]:
import torch
import numpy as np
from torch import nn
import math

u = np.load("/home/user/project/python/rmsKit/array/KH/3site/sel/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_1/u/0.npy")
device = torch.device("cuda")

def random_unitary_matrix(size, device):
    random_matrix = np.random.randn(size, size)
    q, _ = np.linalg.qr(random_matrix)
    return torch.tensor(u, dtype=torch.float64, device=device)

class CustomModel(nn.Module):
    def __init__(self, H_size, unitary_size):
        super(CustomModel, self).__init__()

        # Calculate the number of unitary matrices required to match the size of 'H'
        n_us = round(math.log2(H_size) / math.log2(unitary_size))
        self.n_us = n_us

        # Initialize the given number of random unitary matrices
        self.us = nn.ParameterList([nn.Parameter(random_unitary_matrix(unitary_size, device), requires_grad=True) for _ in range(n_us)])

    def forward(self):
        # Calculate U as the Kronecker product of all the unitary matrices in 'us'
        U = self.us[0]
        U = torch.kron(U, self.us[1])
        U = torch.kron(U, self.us[2])
        U = torch.kron(U, self.us[3])
        # for i in range(4):
        #     U = torch.kron(U, self.us[i+1])
        # for u in self.us[1:]:
        #     U = torch.kron(U, u)
        return U

def custom_loss(U, H):
    # Calculate U @ H @ U.T
    A = torch.matmul(U, torch.matmul(H, U.T))

    # Calculate the absolute value of the result
    a = torch.max(torch.diag(A)) * torch.eye(A.shape[0], device=device)
    result_abs = -torch.abs(A - a) + a

    # Calculate the minimum eigenvalue
    E = torch.linalg.eigvalsh(result_abs)
    z = torch.exp(-E * 1).sum()
    return torch.log(z)

p = dict(
    Jx=1,
    Jy=1,
    Jz=1,
    hx=0,
    hz=0,
)
H = KH.system([2, 2], "3site", p)

# Create an instance of the CustomModel class
model = CustomModel(H.shape[0], 8).to(device)
compiled_model = torch.compile(model, dynamic = False, fullgraph=True)
# Calculate the output
